# Hotdog / Not Hotdog Classification - transfer learning

https://drive.google.com/file/d/1FZ3ZwcPDoEave_xp50Ziue39gMhGPO1W/view

## Imports

In [1]:
from math import ceil

import numpy as np

import tensorflow as tf

tf.logging.set_verbosity(tf.logging.ERROR)

from keras.applications import vgg16
from keras.applications.vgg16 import VGG16
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense
from keras.optimizers import SGD, RMSprop
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


## Using the bottleneck features of a pre-trained network

In [2]:
vgg_model = VGG16(include_top=False, weights="imagenet")
print(vgg_model.summary())

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [3]:
TRAIN_DATA_DIR = "data/train"
VALIDATION_DATA_DIR = "data/test"
BOTTLENECK_FEATURES_TRAIN_PATH = "auxiliary/bottleneck_features_train.npy"
BOTTLENECK_FEATURES_VALIDATION_PATH = "auxiliary/bottleneck_features_validation.npy"
TOP_MODEL_WEIGHTS_PATH = "auxiliary/bottleneck_fc_model.h5"
TUNED_VGG_WEIGHTS_PATH = "auxiliary/tuned_vgg_model.h5"

TRAIN_CLASS_SIZE = 249
VALIDATION_CLASS_SIZE = 250

IMG_WIDTH, IMG_HEIGHT = 150, 150
BATCH_SIZE = 16

In [4]:
def get_data_generator():
    datagen = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input)
    return datagen


def get_batches(
    path,
    datagen=get_data_generator(),
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=True,
    save_to_dir=None,
):
    generator = datagen.flow_from_directory(
        path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode,
        shuffle=shuffle,
        save_to_dir=save_to_dir,
    )
    return generator

We will run our loaded VGG16 model on our training and validation data once, recording the output (the "bottleneck features" from the VGG16 model: the last activation maps before the fully-connected layers) in two numpy arrays. Then we will train a small fully-connected model on top of the stored features.

Below `class_mode=None` means our generator will only yield batches of data without labels, and `shuffle=False` means our data will be in order, so first images will be hotdogs, then not hotdogs. We have exact same number of objects in classes both on train and validation, so here we set `batch_size` to class size and fit generator data in two steps. We do this to use all our training and validation samples by having sample size be multiple of class size, which we wouldn't be able to do if we were using some conventional batch size of 16 or 32 here.

In [5]:
train_generator = get_batches(
    TRAIN_DATA_DIR,
    batch_size=TRAIN_CLASS_SIZE,
    class_mode=None,
    shuffle=False,
    save_to_dir="auxiliary/preview_train",
)
bottleneck_features_train = vgg_model.predict_generator(train_generator, steps=2)
np.save(open(BOTTLENECK_FEATURES_TRAIN_PATH, "wb"), bottleneck_features_train)

validation_generator = get_batches(
    VALIDATION_DATA_DIR,
    batch_size=VALIDATION_CLASS_SIZE,
    class_mode=None,
    shuffle=False,
    save_to_dir="auxiliary/preview_validation",
)
bottleneck_features_validation = vgg_model.predict_generator(validation_generator, steps=2)
np.save(open(BOTTLENECK_FEATURES_VALIDATION_PATH, "wb"), bottleneck_features_validation)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


We'll create a function for generation of our top fully-connected model since we'll be reusing it later.

In [6]:
def generate_top_model(input_shape):
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation="sigmoid"))
    return model

In [7]:
train_data = np.load(open(BOTTLENECK_FEATURES_TRAIN_PATH, "rb"))
# the features were saved in order, so recreating the labels is easy
train_labels = np.array([0] * TRAIN_CLASS_SIZE + [1] * TRAIN_CLASS_SIZE)

validation_data = np.load(open(BOTTLENECK_FEATURES_VALIDATION_PATH, "rb"))
validation_labels = np.array([0] * VALIDATION_CLASS_SIZE + [1] * VALIDATION_CLASS_SIZE)

model = generate_top_model(input_shape=train_data.shape[1:])
model.compile(
    optimizer="rmsprop",
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

In [8]:
# es = EarlyStopping(monitor="val_loss", mode="min", patience=5, verbose=1)

# reduce_lr = ReduceLROnPlateau(
#     monitor="val_acc", patience=5, verbose=1, factor=0.5, min_lr=1e-4
# )

checkpoint = ModelCheckpoint(
    TOP_MODEL_WEIGHTS_PATH,
    monitor="val_acc",
    verbose=1,
    save_best_only=True,
    mode="max",
)

model.fit(
    train_data,
    train_labels,
    epochs=200,
    batch_size=BATCH_SIZE,
    validation_data=(validation_data, validation_labels),
    callbacks=[checkpoint],
)

Train on 498 samples, validate on 500 samples
Epoch 1/200
498/498 [==============================] - 1s 3ms/step - loss: 6.0551 - acc: 0.6024 - val_loss: 5.8707 - val_acc: 0.6160

Epoch 00001: val_acc improved from -inf to 0.61600, saving model to auxiliary/bottleneck_fc_model.h5
Epoch 2/200
498/498 [==============================] - 1s 2ms/step - loss: 4.1459 - acc: 0.7289 - val_loss: 4.3839 - val_acc: 0.7120

Epoch 00002: val_acc improved from 0.61600 to 0.71200, saving model to auxiliary/bottleneck_fc_model.h5
Epoch 3/200
498/498 [==============================] - 1s 2ms/step - loss: 3.8449 - acc: 0.7430 - val_loss: 3.3066 - val_acc: 0.7740

Epoch 00003: val_acc improved from 0.71200 to 0.77400, saving model to auxiliary/bottleneck_fc_model.h5
Epoch 4/200
498/498 [==============================] - 1s 2ms/step - loss: 3.5470 - acc: 0.7711 - val_loss: 2.9035 - val_acc: 0.8040

Epoch 00004: val_acc improved from 0.77400 to 0.80400, saving model to auxiliary/bottleneck_fc_model.h5
Epoch

## Fine-tuning the top layers of a a pre-trained network

After instantiating the VGG base and loading its weights, we add our previously trained fully-connected classifier on top. We start with a fully-trained classifier, including the top classifier, in order to successfully do fine-tuning. 

In [9]:
base_model = VGG16(
    include_top=False, weights="imagenet", input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
)

top_model = generate_top_model(input_shape=(base_model.output_shape[1:]))
top_model.load_weights(TOP_MODEL_WEIGHTS_PATH)

# add the model on top of the convolutional base
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

We'll be freezing all convolutional layers up to the last convolutional block and only fine tune the last one which has more specialized features. Let's take a look at our model's layers to freeze the right amount.

In [10]:
model.layers

In [11]:
# freeze the first 15 layers (up to the last conv block)
for layer in model.layers[:15]:
    layer.trainable = False

model.compile(
    loss="binary_crossentropy",
    optimizer=RMSprop(lr=3e-6),
    metrics=["accuracy"],
)

Let's check that the last convolutional block and our top fully-connected model are trainable.

In [12]:
for layer in model.layers:
    print(f"{layer.name}\t{layer.trainable}")

input_2	False
block1_conv1	False
block1_conv2	False
block1_pool	False
block2_conv1	False
block2_conv2	False
block2_pool	False
block3_conv1	False
block3_conv2	False
block3_conv3	False
block3_pool	False
block4_conv1	False
block4_conv2	False
block4_conv3	False
block4_pool	False
block5_conv1	True
block5_conv2	True
block5_conv3	True
block5_pool	True
sequential_2	True


In [13]:
train_generator = get_batches(TRAIN_DATA_DIR)
validation_generator = get_batches(VALIDATION_DATA_DIR)

checkpoint = ModelCheckpoint(
    TUNED_VGG_WEIGHTS_PATH,
    monitor="val_acc",
    verbose=1,
    save_best_only=True,
    mode="max",
)

model.fit_generator(
    train_generator,
    steps_per_epoch=ceil(TRAIN_CLASS_SIZE * 2 / BATCH_SIZE),
    epochs=50,
    validation_data=validation_generator,
    validation_steps=ceil(VALIDATION_CLASS_SIZE * 2 / BATCH_SIZE),
    callbacks=[checkpoint],
)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
Epoch 1/50
32/32 [==============================] - 86s 3s/step - loss: 0.6514 - acc: 0.9589 - val_loss: 2.2564 - val_acc: 0.8560

Epoch 00001: val_acc improved from -inf to 0.85600, saving model to auxiliary/tuned_vgg_model.h5
Epoch 2/50
32/32 [==============================] - 79s 2s/step - loss: 0.8262 - acc: 0.9472 - val_loss: 2.3329 - val_acc: 0.8480

Epoch 00002: val_acc did not improve from 0.85600
Epoch 3/50
32/32 [==============================] - 77s 2s/step - loss: 0.6451 - acc: 0.9570 - val_loss: 2.2611 - val_acc: 0.8580

Epoch 00003: val_acc improved from 0.85600 to 0.85800, saving model to auxiliary/tuned_vgg_model.h5
Epoch 4/50
32/32 [==============================] - 74s 2s/step - loss: 0.5632 - acc: 0.9648 - val_loss: 2.1194 - val_acc: 0.8580

Epoch 00004: val_acc did not improve from 0.85800
Epoch 5/50
32/32 [==============================] - 75s 2s/step - loss: 0.5564 - acc: 0.9629 - va

With transfer learning we were able to increase accuracy to apprx 86% on the validation set comparing to 70% for our custom convnet model.

Comparing fine tuning of the last convolutional block to training only fully connected model on top of VGG16 convolutional base, the increase in accuracy for the best checkpointed model is not that high - around 1%, but the values validation accuracy is varied around during training went from 82-83% to 85-86%.